# 中国选取211 985

In [42]:
import pandas as pd
import os
from shutil import copyfile
import re

In [8]:
df = pd.read_excel('./211 985.xlsx', names = ['names'])

In [2]:
df = pd.read_excel('./CSRanking_USA.xlsx')

In [6]:
os.chdir('./data_with_class/USA')
os.getcwd()

'/home/gs/git-repos/University-Logo-Deep-Learning-Community-Detection/data_with_class/USA'

In [22]:
for sch in df.names:
    try:
        copyfile(sch + '_China.jpeg', '../../selected_data/China/' + sch + '_China.jpeg')
    except:
        print(sch)

第二军医大学 
中国石油大学
中国地质大学
国防科学技术大学
第四军医大学
新疆大学


# 重新爬取CSRanking 上USA前162名的学校的校徽

In [15]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import html5lib
from urllib.request import urlretrieve

In [14]:
keywords = '{school} seal'

In [18]:
def getUrl(browser, universityName, keywords):
    browser.get('https://www.google.com/imghp')
    google_input = browser.find_element_by_class_name('gLFyf')
    google_input.send_keys(keywords.format(school = universityName))
    google_input.send_keys(Keys.ENTER)
    first_result = browser.find_element_by_css_selector('#rg img')
    src = first_result.get_attribute('src')
    return src

def downloadImg(url, directory, name, datatype):
    from urllib.request import urlretrieve
    urlretrieve(url, directory + name + '.' + datatype)

def main(keywords, country, universityList):
    count = 0
    browser = webdriver.Chrome()
    for school in universityList:
        url = ''
        if count == 15:
            count = 0
            browser.close()
            browser = webdriver.Chrome()
        count += 1
        #try:
        url = getUrl(browser, school, keywords)
        datatype = re.findall(r'image/.+?;', url)[0][6:-1]
        downloadImg(url, 'selected_data/{}/'.format(country), school, datatype)
        #except:
        print('\nUniversity :' + school + '    failed\n')
        continue
        #print('University : ' + school + '     Success')

In [ ]:
main(keywords, 'USA', [x.strip() for x in df.Institution])

In [22]:
os.chdir('../../')

In [28]:
os.getcwd()

'/home/gs/git-repos/University-Logo-Deep-Learning-Community-Detection/selected_data/USA'

# 处理得到的数据

In [26]:
from PIL import Image, ImageChops

def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

In [27]:
os.chdir('./selected_data/USA')

In [41]:
for sch in os.listdir():
    im = Image.open(sch)
    result = trim(im)
    #if sch.endswith('png') or im.format == 'JPEG' or im.format == 'JPG':
    result = result.convert('RGB') # 转换格式为RGB
    result = result.resize((100, 100)) # resize
    os.remove(sch) # 删除原有图片
    result.save(re.findall('.+\.', sch)[0][:-1] + '_USA.jpeg')
    